In [8]:
import os, sys
import numpy as np
import imageio
import json
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm, trange

import matplotlib.pyplot as plt

from run_nerf_helpers import *
from ray_utils import *
from layers.point_sampling import *
from layers.camera_transform import *

from load_llff import load_llff_data
from load_deepvoxels import load_dv_data
from load_blender import load_blender_data

In [17]:
datadir = './llff_orchid/orchids'
print(os.path.abspath(datadir))
images, poses, bds, planes, render_poses, i_test = load_llff_data(
    datadir, plane_name='',
    factor=8, recenter=True, bd_factor=.75, spherify=False
)
hwf = poses[0,:3,-1]
poses = poses[:,:3,:4]
print('Loaded llff', images.shape, render_poses.shape, hwf, datadir)

i_test = np.arange(images.shape[0])[::8]

i_val = i_test
i_train = np.array([i for i in np.arange(int(images.shape[0])) if
                (i not in i_test and i not in i_val)])

print(i_train, i_test)

print('DEFINING BOUNDS')
near = np.ndarray.min(bds) * .9
far = np.ndarray.max(bds) * 1.

print('NEAR FAR', near, far)

/data/new_disk/wangph/nerf-pytorch/data/nerf_llff_data/orchids
Loaded image data (378, 504, 3, 25) [378.         504.         390.57784585]
Loaded ./data/nerf_llff_data/orchids 14.48833973008133 124.51196423745613
recentered (3, 5)
[[ 1.00000000e+00  1.68170561e-10 -3.61509422e-09  5.96046434e-10]
 [-1.68170602e-10  1.00000000e+00 -1.26528299e-08 -1.07288365e-08]
 [ 3.61509422e-09  1.26528299e-08  1.00000000e+00 -2.98023217e-10]]
Data:
(25, 3, 5) (25, 378, 504, 3) (25, 2)
HOLDOUT view is 12
Loaded llff (25, 378, 504, 3) (120, 3, 5) [378.      504.      390.57785] ./data/nerf_llff_data/orchids
[ 1  2  3  4  5  6  7  9 10 11 12 13 14 15 17 18 19 20 21 22 23] [ 0  8 16 24]
DEFINING BOUNDS
NEAR FAR 1.200000035762787 11.458590507507324


In [5]:
# Cast intrinsics to right types
H, W, focal = hwf
H, W = int(H), int(W)
hwf = [H, W, focal]

In [9]:
# Prepare raybatch tensor if batching random rays
def prepare_batch(i_sel):
    N_rand = 1024
    no_camera_id = False

    # For random ray batching
    train_poses, train_imgs = poses[i_sel], images[i_sel]

    rays = np.stack([get_rays_np(H, W, focal, p) for p in train_poses[:,:3,:4]], 0) # [N, ro+rd, H, W, 3]
    print('done, concats', rays.shape, train_imgs[:,None,...].shape)

    rays_rgb = np.concatenate([rays, train_imgs[:,None,...]], 1) # [N, ro+rd+rgb, H, W, 3]
    rays_rgb = np.transpose(rays_rgb, [0,2,3,1,4]) # [N, H, W, ro+rd+rgb, 3]
    rays_rgb = rays_rgb.astype(np.float32)

    print('done', rays_rgb.shape)

    return rays_rgb

In [10]:
rays_rgb = prepare_batch(i_train)
rays = rays_rgb[..., :2, :]
rgbs = rays_rgb[..., 2, :]
print(rays.shape, rgbs.shape)

done, concats (21, 2, 378, 504, 3) (21, 1, 378, 504, 3)
done (21, 378, 504, 3, 3)
(21, 378, 504, 2, 3) (21, 378, 504, 3)


In [16]:
print("Check rays size:", rays.shape)
output_path = os.path.join(datadir, 'rays_train.npy')
np.save(output_path, rays)
print("Save to training set to %s" % output_path)

print("Check rgbs size:", rgbs.shape)
output_path = os.path.join(datadir, 'rgbs_train.npy')
np.save(output_path, rgbs)
print("Save to training set to %s" % output_path)

Check rays size: (4, 378, 504, 2, 3)
Save to training set to ./data/nerf_llff_data/orchids/rays_train.npy
Check rgbs size: (4, 378, 504, 3)
Save to training set to ./data/nerf_llff_data/orchids/rgbs_train.npy


In [12]:
rays_rgb = prepare_batch(i_test)
rays = rays_rgb[..., :2, :]
rgbs = rays_rgb[..., 2, :]
print(rays.shape, rgbs.shape)

done, concats (4, 2, 378, 504, 3) (4, 1, 378, 504, 3)
done (4, 378, 504, 3, 3)
(4, 378, 504, 2, 3) (4, 378, 504, 3)


In [13]:
print("Check rays size:", rays.shape)
output_path = os.path.join(datadir, 'rays_test.npy')
np.save(output_path, rays)
print("Save to testing set to %s" % output_path)

print("Check rgbs size:", rgbs.shape)
output_path = os.path.join(datadir, 'rgbs_test.npy')
np.save(output_path, rgbs)
print("Save to testing set to %s" % output_path)

Check rays size: (4, 378, 504, 2, 3)
Save to testing set to ./data/nerf_llff_data/orchids/rays_test.npy
Check rgbs size: (4, 378, 504, 3)
Save to testing set to ./data/nerf_llff_data/orchids/rgbs_test.npy


In [14]:
render_rays = np.stack([get_rays_np(H, W, focal, p) for p in render_poses[:,:3,:4]], 0) # [N, ro+rd, H, W, 3]
render_rays = np.transpose(render_rays, [0,2,3,1,4]) # [N, H, W, ro+rd, 3]
print('done, concats', render_rays.shape)

print("Check rays size:", render_rays.shape)
output_path = os.path.join(datadir, 'rays_exhibit.npy')
np.save(output_path, render_rays)
print("Save to testing set to %s" % output_path)

done, concats (120, 378, 504, 2, 3)
Check rays size: (120, 378, 504, 2, 3)
Save to testing set to ./data/nerf_llff_data/orchids/rays_exhibit.npy


In [15]:
import json
meta_dict = {
    'near': 1.200000035762787,
    'far': 11.458590507507324
}

output_path = os.path.join(datadir, 'meta.json')
with open(output_path, 'w') as f:
    json.dump(meta_dict, f)
    print("Save to meta data to %s" % output_path)

Save to meta data to ./data/nerf_llff_data/orchids/meta.json
